In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import soccerdata as sd
from pathlib import PosixPath
from gde_utils.date_utils import to_datetime
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

[03/03/24 21:51:48] INFO     Custom team name replacements loaded from                                ]8;id=715590;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=877133;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#85\85]8;;\
                             /home/morten/soccerdata/config/teamname_replacements.json.                            

                    INFO     Custom league dict loaded from                                          ]8;id=840004;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=130682;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#163\163]8;;\
                             /home/morten/soccerdata/config/league_dict.json.                                      

In [ ]:
# load all data
ws = sd.WhoScored(
    leagues="GER-Bundesliga",
    seasons=[15, 16, 17, 18, 19, 20, 21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=False,
)

In [ ]:
buli_schedule = ws.read_schedule()
elo = sd.ClubElo()
buli_schedule.reset_index(inplace=True)

In [ ]:
def is_own_goal(qualifiers):
    return [max([(True if x["type"]["displayName"] == "OwnGoal" else False) for x in events], default=False) for events in qualifiers]

def get_opposition_team(df_goals: pd.DataFrame, df_teams: pd.DataFrame):
    switched_teams = df_goals.copy()
    team_id_one = df_teams["team_id"].unique()[0]
    team_id_two = df_teams["team_id"].unique()[1]
    switched_teams.replace({team_id_one : team_id_two,
                            team_id_two : team_id_one}, inplace = True)
    return switched_teams

def get_score(events_df: pd.DataFrame, df_teams: pd.DataFrame):
    goals = events_df.loc[(events_df["is_goal"] == True)].copy()
    goals["own_goal"] = is_own_goal(goals["qualifiers"])
    goals.loc[~goals["own_goal"], "goal_team_id"] = goals.loc[~goals["own_goal"],"team_id"]
    goals.loc[goals["own_goal"], "goal_team_id"] = get_opposition_team(goals["team_id"], df_teams)[goals["own_goal"]]
    goals.reset_index(inplace=True)
    return goals[["expanded_minute", "goal_team_id"]]

In [ ]:
data = []
failed_count = 0
# create dataset:
for game_id in buli_schedule.game_id.values:
    try:
        # pre match information:
        pre_game_date = to_datetime(buli_schedule[buli_schedule["game_id"] == game_id]["date"].values[0]) - timedelta(days=1)
        game_elo = elo.read_by_date(pre_game_date)
        game_elo.reset_index(inplace=True)
        home_team = buli_schedule[buli_schedule["game_id"] == game_id].home_team.values[0]
        away_team = buli_schedule[buli_schedule["game_id"] == game_id].away_team.values[0]
        home_elo = game_elo[game_elo["team"] == home_team].elo.values[0]
        away_elo = game_elo[game_elo["team"] == away_team].elo.values[0]
        df_events = ws.read_events([game_id])
        loader = ws.read_events(match_id=[game_id], output_fmt='loader')
        df_teams = loader.teams(game_id)
        df_events.reset_index(inplace=True)
        game_score = get_score(df_events, df_teams)
        score_readable = game_score.groupby("goal_team_id").count().reset_index().rename(columns={"expanded_minute": "score"})
        score_value_home = score_readable[score_readable["goal_team_id"] == (home_id := df_teams[df_teams["team_name"] == home_team].team_id.values[0])].score.values
        score_value_away = score_readable[score_readable["goal_team_id"] == (away_id := df_teams[df_teams["team_name"] == away_team].team_id.values[0])].score.values
        home_score = score_value_home[0] if len(score_value_home) > 0 else 0
        away_score = score_value_away[0] if len(score_value_away) > 0 else 0
        data.append([home_id, home_score, home_elo, away_id, away_score, away_elo])
    except:
        failed_count +=1
print("Failed: ", failed_count)

In [ ]:
print(failed_count)

In [ ]:
elo_dif = pd.DataFrame(data, columns=["home_id", "home_score", "home_elo", "away_id", "away_score", "away_elo"])

In [33]:
elo_dif = pd.read_csv("metrics/mov_elo/elo_diff.csv", sep=";")

In [39]:
elo_dif["minutes"] = 90

In [40]:
elo_dif2 = elo_dif.copy()
elo_dif2["home_elo"] = elo_dif["away_elo"]
elo_dif2["away_elo"] = elo_dif["home_elo"]
elo_dif2["home_score"] = elo_dif["away_score"]
elo_dif2["away_score"] = elo_dif["home_score"]

elo_dif3 = pd.concat([elo_dif, elo_dif2])

In [41]:
# get elo and margin of victory
elo_dif3["elo_dif"] = elo_dif3["home_elo"] - elo_dif3["away_elo"]
elo_dif3["goal_dif"] = elo_dif3["home_score"] - elo_dif3["away_score"]



In [42]:
elo_dif.corr()

,home_id,home_score,home_elo,away_id,away_score,away_elo,elo_dif,goal_dif,minutes
home_id,1.000000,0.037339,0.048797,-0.053437,-0.036929,0.002702,0.032253,0.048946,NaN
home_score,0.037339,1.000000,0.331736,-0.063978,-0.148998,-0.175426,0.354842,0.789880,NaN
home_elo,0.048797,0.331736,1.000000,-0.002790,-0.137979,-0.021437,0.714697,0.316949,NaN
away_id,-0.053437,-0.063978,-0.002790,1.000000,0.054873,0.045647,-0.033887,-0.078654,NaN
away_score,-0.036929,-0.148998,-0.137979,0.054873,1.000000,0.266768,-0.283173,-0.724106,NaN
away_elo,0.002702,-0.175426,-0.021437,0.045647,0.266768,1.000000,-0.714594,-0.287800,NaN
elo_dif,0.032253,0.354842,0.714697,-0.033887,-0.283173,-0.714594,1.000000,0.423113,NaN
goal_dif,0.048946,0.789880,0.316949,-0.078654,-0.724106,-0.287800,0.423113,1.000000,NaN
minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from database_io.db_player import DB_player

In [1]:
from sklearn.linear_model import LinearRegression
import json
from database_io import DB_player
import numpy as np
        # get data
db = DB_player("GDE.db")
df = db.get_all_games()
df["elo_diff"] = df["elo"] - df["opposition_elo"]
df["diff"] = df["result"].apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]))

[03/04/24 18:10:40] INFO     Custom team name replacements loaded from                                ]8;id=470527;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=212801;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#85\85]8;;\
                             /home/morten/soccerdata/config/teamname_replacements.json.                            

                    INFO     Custom league dict loaded from                                          ]8;id=463813;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=792249;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#163\163]8;;\
                             /home/morten/soccerdata/config/league_dict.json.                                      

In [5]:
db = DB_player("GDE.db")
df = db.get_all_games()
df["elo_diff"] = df["elo"] - df["opposition_elo"]
df["diff"] = df["result"].apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]))

In [6]:
# create regression model
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(np.array([df["elo_diff"].values, df["minutes"].values]).T, df["diff"].values.reshape(-1, 1))

LinearRegression()

In [7]:
# elo_dif.to_csv("elo_diff.csv", index=False, sep=";")'

In [8]:
lr.predict([[-158, 90]])

array([[-0.61084105]])

In [9]:
lr.intercept_[0]

-0.14054139912744046

In [10]:
lr.coef_[0][0]

0.0040318842488232184

In [11]:
lr.coef_[0][1]

0.0018526451219410477